In [17]:
import sys
import os
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import json
import joblib
from pprint import pprint
current_path = os.getcwd()
project_root = os.path.abspath(os.path.join(current_path, os.pardir, os.pardir))
print(project_root)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

c:\Users\benhu\UCL\Term 3\HSBC\deep-lob-project


In [21]:
loaded = joblib.load(r"C:\Users\benhu\UCL\Term 3\HSBC\deep-lob-project\results\algo\AAPL\2025-06-05\20\15\DeepLOB\data_uQD6pqDxul.joblib")
pprint(loaded)

{'directions': array([0., 0., 0., ..., 0., 0., 1.]),
 'lower_thresh': -0.18408013880252838,
 'meta': {'date': '2025-06-05',
          'horizon': 20,
          'modelClass': <class 'src.algo.algoModels.deepLOB.DeepLOB'>,
          'plot': True,
          'representation': 'orderflows',
          'rowLim': 10000,
          'saveResults': True,
          'signalPercentage': 15,
          'ticker': 'AAPL',
          'verbose': False,
          'windowLength': 100},
 'pnl': array([0.   , 0.   , 0.   , ..., 0.065, 0.065, 0.065]),
 'predictions': array([0.39599341, 0.23789294, 0.33941114, ..., 0.29923248, 0.23469584,
       0.43407077]),
 'upper_thresh': 0.432226076722145}
